In [1]:
import sddk
import pandas as pd
pd.options.display.max_columns = 1000 # to see all columns
import json
import re
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

In [2]:
EDHCSg = sddk.read_file("EDHCSg.geojson", "gdf", "a9237c5ea642d4714bcdefb03f70a1f4")

reading file located in a public folder


In [3]:
import relate

In [4]:
len(EDHCSg)

137305

In [15]:
print(EDHCSg.columns.tolist())

['EDCS-ID', 'publication', 'province', 'province_list', 'place', 'place_list', 'end_yr_list', 'notes_dating', 'status_list', 'inscr_type', 'status_notation', 'inscr_process', 'notes_references', 'notes_comments', 'inscription', 'inscription_stripped_final', 'Links', 'dating from', 'dating to', 'status', 'Latitude', 'Longitude', 'photo', 'Material', 'Comment', 'EDH-ID', 'language_EDCS', 'clean_text_interpretive_word_EDCS', 'responsible_individual', 'type_of_inscription', 'letter_size', 'literature', 'work_status', 'height', 'diplomatic_text', 'people', 'depth', 'material', 'type_of_monument', 'province_label', 'width', 'transcription', 'country', 'uri', 'findspot_ancient', 'last_update', 'modern_region', 'findspot_modern', 'edh_geography_uri', 'commentary', 'trismegistos_uri', 'external_image_uris', 'fotos', 'idno_tm', 'placenames_refs', 'text_edition', 'origdate_text', 'layout_execution', 'layout_execution_text', 'support_objecttype', 'support_objecttype_text', 'support_material', 'sup

In [20]:
### subselect votive inscriptions
EDHCSg_votive = EDHCSg[EDHCSg["type_of_inscription_clean"]=="votive inscription"]
len(EDHCSg_votive)

12328

In [24]:
n = 10
texts = tuple(EDHCSg_votive["clean_text_interpretive_word"].tolist()[:n])
all_texts1, all_texts2 = texts, texts
names = EDHCSg_votive["EDH-ID"].tolist()[:n]

In [31]:
import importlib
importlib.reload(relate)

<module 'relate' (namespace)>

In [33]:
from relate import shingle, metrics, matrix
shingle.length = 2
similarity = metrics.select['sørensen_dice']
# Without standardizing function: 'similarity','distance'. With the function: 'st_similarity', 'st_distance' 
matrix = matrix.select['distance']
result = matrix(similarity,all_texts1, all_texts2)
print(result)

NameError: name 'names' is not defined